In [ ]:
!pip install transformers

In [ ]:
from transformers import BartTokenizer, BartModel
import torch
from sklearn.metrics import classification_report
from tqdm import tqdm
import numpy as np
import pandas as pd
import pickle

In [ ]:

tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')
print("bart model is available")
model = BartModel.from_pretrained('facebook/bart-large')
model = model.to(torch.device('cpu'))
model.eval()


In [ ]:
from google.colab import drive
drive.mount("/content/drive",force_remount=True)

In [ ]:
cd drive/MyDrive/MTP_Research/Dataset

In [ ]:
df_sarc = pd.read_csv('mustard++_sarcastic_utterances.csv')

In [ ]:
print("Tokenizer and model initialized")

def get_text_embeddings(text):
    inputs = tokenizer(text, return_tensors="pt")
    outputs = model.encoder(**inputs,output_hidden_states=True)
    hidden_states = outputs.hidden_states
    token_vecs = torch.cat((hidden_states[-4], torch.cat((hidden_states[-3], torch.cat(
        (hidden_states[-2], hidden_states[-1]), dim=0)), dim=0)), dim=0)
    bart_emb = torch.mean(torch.mean(token_vecs, dim=0), dim=0)
    return bart_emb


# For Merging

In [ ]:

# bart_text_features = {}
# # path = '/home/development/apoorvan/AN_MTP/SER/data/'

# combined_sarc = pd.read_csv('mustard++_sarcasm_detection.csv')
# sarc_utterances = pd.read_csv('mustard_PP_final_sarcastic_utterance.csv')
# sarc_utterances = sarc_utterances[['SCENE']]


In [ ]:
# df_sarc = pd.merge(combined_sarc,sarc_utterances,on='SCENE',how='inner')    

In [ ]:

# bart_text_features = {}
df_sarc = pd.read_csv('mustard++_sarcastic_utterances.csv')
# sarc_utterances = pd.read_csv('mustard_PP_final_sarcastic_utterance.csv')
# sarc_utterances = sarc_utterances[['SCENE']]

In [ ]:
df_sarc.head()

In [ ]:
bart_text = {}
bart_keys = []
bart_vals = []
for i in tqdm(range(len(df_sarc))):
    key = df_sarc.loc[i,'KEY']
    if key.find("_u")!=-1:
      bart_text[key] = df_sarc.loc[i, 'SENTENCE']
    else:
      if df_sarc.loc[i, 'SCENE']+"_c" in bart_text.keys():
          bart_text[df_sarc.loc[i, 'SCENE']+"_c"]+= df_sarc.loc[i, 'SENTENCE']
      else:
          bart_text[df_sarc.loc[i, 'SCENE']+"_c"] = df_sarc.loc[i, 'SENTENCE']

bart_keys = list(bart_text.keys())
bart_values = list(bart_text.values())
    
print(len(bart_keys))
# print(bart_values[:5])

In [ ]:
# len(set(bart_keys))

In [ ]:
df = pd.DataFrame(list(zip(bart_keys,bart_values)),columns=['KEY','SENTENCE'])
# print(df.head())

In [ ]:
df.to_csv('mustard++_all_cu.csv', index=False)

# Sentences to Features

In [ ]:
bart_text_features = {}
for i in range(len(df['KEY'])):
  # print(i,"--",bart_text[i])
  bart_text_features[df.loc[i,'KEY']] = get_text_embeddings(
        df.loc[i,'SENTENCE']).to(torch.device('cpu')).detach().numpy()

In [ ]:
pickle.dump(bart_text_features, open(
    'aug_text_features_bart.pickle', 'wb'))

# Into the Drive

In [ ]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [ ]:
# cp /content/text_features_meanbart.pickle '/content/gdrive/My Drive/MTP Research/Dataset'